In [1]:
import os
import pandas as pd

In [2]:
res = pd.read_csv('../results/eurosat_probing_rebuttal1.csv', index_col = 0)
res['Data Set'] = 'EuroSat'
res['Dimension'] = res['N Shots'] - 1
all_results = res

datasets = {
            'Flowers102': 'flowers',
            'CIFAR-10': 'cifar10',
            # 'PatternNet': 'patternnet'
            # 'UCMerced': 'ucmerced',
            # 'PatternNet': 'patternnet'
            }

base_path = '../results'
suffix = '_probing_rebuttal1.csv'
for d_name in datasets.keys():
    d_id = datasets[d_name]
    path = os.path.join(base_path,f'{d_id}{suffix}')
    res = pd.read_csv(path, index_col = 0)
    res['Data Set'] = d_name
    if d_id in ['flowers', 'cifar10']:
        res['Dimension'] = 1
    else:
        res['Dimension'] = res['N Shots'] - 1
    all_results = pd.concat([all_results, res])

In [3]:


# Group by 'N Shots', 'Method', and 'Data Set' and calculate mean and std
summary_df = all_results.groupby(['N Shots', 'Method', 'Data Set', 'Dimension'])['Accuracy'].agg(['mean', 'std']).reset_index()

# Rename columns for clarity
summary_df.rename(columns={'mean': 'Accuracy Mean', 'std': 'Accuracy Std'}, inplace=True)

summary_df = summary_df.sort_values(by = ['Data Set', 'N Shots', 'Method'])

# Display the result
print(summary_df)


    N Shots        Method    Data Set  Dimension  Accuracy Mean  Accuracy Std
0         3      FlagNets    CIFAR-10          1        0.59601      0.010055
3         3     ProtoNets    CIFAR-10          1        0.58608      0.008960
6         3  SubspaceNets    CIFAR-10          1        0.59634      0.009873
9         5      FlagNets    CIFAR-10          1        0.65173      0.009121
12        5     ProtoNets    CIFAR-10          1        0.65161      0.009495
15        5  SubspaceNets    CIFAR-10          1        0.65184      0.009188
18        7      FlagNets    CIFAR-10          1        0.67999      0.007455
21        7     ProtoNets    CIFAR-10          1        0.68634      0.007746
24        7  SubspaceNets    CIFAR-10          1        0.68064      0.007256
1         3      FlagNets     EuroSat          2        0.77661      0.010250
4         3     ProtoNets     EuroSat          2        0.76708      0.009521
7         3  SubspaceNets     EuroSat          2        0.77578 

In [4]:
# Define the custom order for 'Data Set'
data_set_order = ['EuroSat', 'CIFAR-10', 'Flowers102']

# Convert 'Data Set' to a categorical type with the specified order
summary_df['Data Set'] = pd.Categorical(summary_df['Data Set'], categories=data_set_order, ordered=True)


# for n_shots in summary_df['N Shots'].unique():
#     idx = summary_df['N Shots'] == n_shots
pivot_df = summary_df.pivot_table(
    columns= 'Method',
    index= ['N Shots', 'Data Set'],
    values='Accuracy Mean'
)
pivot_df = pivot_df.round(4)
pivot_df = pivot_df.applymap(lambda x: f"{x:.4f}".rstrip('0').rstrip('.') if pd.notnull(x) else '')
print(pivot_df)
print(pivot_df.to_latex(index=True))

Method             FlagNets ProtoNets SubspaceNets
N Shots Data Set                                  
3       EuroSat      0.7766    0.7671       0.7758
        CIFAR-10      0.596    0.5861       0.5963
        Flowers102   0.9018    0.8816       0.9017
5       EuroSat      0.8182    0.8069       0.8176
        CIFAR-10     0.6517    0.6516       0.6518
        Flowers102   0.9325    0.9141       0.9324
7       EuroSat      0.8386    0.8258       0.8385
        CIFAR-10       0.68    0.6863       0.6806
        Flowers102   0.9454    0.9269       0.9452


/tmp/ipykernel_25065/3569597460.py:10: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_df = summary_df.pivot_table(
/tmp/ipykernel_25065/3569597460.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivot_df = pivot_df.applymap(lambda x: f"{x:.4f}".rstrip('0').rstrip('.') if pd.notnull(x) else '')


\begin{tabular}{lllll}
\toprule
 & Method & FlagNets & ProtoNets & SubspaceNets \\
N Shots & Data Set &  &  &  \\
\midrule
\multirow[t]{3}{*}{3} & EuroSat & 0.7766 & 0.7671 & 0.7758 \\
 & CIFAR-10 & 0.596 & 0.5861 & 0.5963 \\
 & Flowers102 & 0.9018 & 0.8816 & 0.9017 \\
\cline{1-5}
\multirow[t]{3}{*}{5} & EuroSat & 0.8182 & 0.8069 & 0.8176 \\
 & CIFAR-10 & 0.6517 & 0.6516 & 0.6518 \\
 & Flowers102 & 0.9325 & 0.9141 & 0.9324 \\
\cline{1-5}
\multirow[t]{3}{*}{7} & EuroSat & 0.8386 & 0.8258 & 0.8385 \\
 & CIFAR-10 & 0.68 & 0.6863 & 0.6806 \\
 & Flowers102 & 0.9454 & 0.9269 & 0.9452 \\
\cline{1-5}
\bottomrule
\end{tabular}



In [5]:
# Define the custom order for 'Data Set'
data_set_order = ['EuroSat', 'CIFAR-10', 'Flowers102']

idx = (summary_df['Data Set'] == 'EuroSat') |\
      (summary_df['Data Set'] == 'CIFAR-10') |\
      (summary_df['Data Set'] == 'Flowers102')
summary_df = summary_df[idx]

# Convert 'Data Set' to a categorical type with the specified order
summary_df['Data Set'] = pd.Categorical(summary_df['Data Set'], categories=data_set_order, ordered=True)

# Multiply the 'Accuracy Mean' and 'Accuracy Std' by 100
summary_df['Accuracy Mean'] *= 100
summary_df['Accuracy Std'] *= 100

# Pivot the table with 'Accuracy Mean' and 'Accuracy Std' separately
pivot_mean = summary_df.pivot_table(
    columns='Method',
    index=['N Shots', 'Data Set'],
    values='Accuracy Mean'
)

pivot_std = summary_df.pivot_table(
    columns='Method',
    index=['N Shots', 'Data Set'],
    values='Accuracy Std'
)

pivot_combined = pivot_mean.copy()  # Start with a copy of the mean table for structure
for col in pivot_combined.columns:
    pivot_combined[col] = [
        f"{mean:.1f} ± {std:.1f}" if pd.notnull(mean) and pd.notnull(std) else ''
        for mean, std in zip(pivot_mean[col], pivot_std[col])
    ]

print(pivot_combined)
print(pivot_combined.to_latex(index=True))

Method                FlagNets   ProtoNets SubspaceNets
N Shots Data Set                                       
3       EuroSat     77.7 ± 1.0  76.7 ± 1.0   77.6 ± 1.0
        CIFAR-10    59.6 ± 1.0  58.6 ± 0.9   59.6 ± 1.0
        Flowers102  90.2 ± 0.7  88.2 ± 1.0   90.2 ± 0.7
5       EuroSat     81.8 ± 0.7  80.7 ± 0.8   81.8 ± 0.7
        CIFAR-10    65.2 ± 0.9  65.2 ± 0.9   65.2 ± 0.9
        Flowers102  93.2 ± 0.5  91.4 ± 0.6   93.2 ± 0.5
7       EuroSat     83.9 ± 0.8  82.6 ± 0.8   83.8 ± 0.8
        CIFAR-10    68.0 ± 0.7  68.6 ± 0.8   68.1 ± 0.7
        Flowers102  94.5 ± 0.5  92.7 ± 0.5   94.5 ± 0.5
\begin{tabular}{lllll}
\toprule
 & Method & FlagNets & ProtoNets & SubspaceNets \\
N Shots & Data Set &  &  &  \\
\midrule
\multirow[t]{3}{*}{3} & EuroSat & 77.7 ± 1.0 & 76.7 ± 1.0 & 77.6 ± 1.0 \\
 & CIFAR-10 & 59.6 ± 1.0 & 58.6 ± 0.9 & 59.6 ± 1.0 \\
 & Flowers102 & 90.2 ± 0.7 & 88.2 ± 1.0 & 90.2 ± 0.7 \\
\cline{1-5}
\multirow[t]{3}{*}{5} & EuroSat & 81.8 ± 0.7 & 80.7 ± 0.8 & 81.8 

/tmp/ipykernel_25065/3211811896.py:17: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_mean = summary_df.pivot_table(
/tmp/ipykernel_25065/3211811896.py:23: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_std = summary_df.pivot_table(


In [6]:
summary_df

,N Shots,Method,Data Set,Dimension,Accuracy Mean,Accuracy Std
0,3,FlagNets,CIFAR-10,1,59.601,1.005532
3,3,ProtoNets,CIFAR-10,1,58.608,0.895965
6,3,SubspaceNets,CIFAR-10,1,59.634,0.987290
9,5,FlagNets,CIFAR-10,1,65.173,0.912101
12,5,ProtoNets,CIFAR-10,1,65.161,0.949492
15,5,SubspaceNets,CIFAR-10,1,65.184,0.918823
18,7,FlagNets,CIFAR-10,1,67.999,0.745498
21,7,ProtoNets,CIFAR-10,1,68.634,0.774627
24,7,SubspaceNets,CIFAR-10,1,68.064,0.725552
1,3,FlagNets,EuroSat,2,77.661,1.025003
